This notebook contains experiments with different libraries for optimization

In [ ]:
!pip install benderopt

In [ ]:
import logging

import numpy as np
from benderopt import minimize

logging.basicConfig(level=logging.ERROR)  # logging.INFO will print less information
logging.getLogger("benderopt").setLevel(logging.ERROR)


# We want to minimize the sinus function between 0 and 2pi
def f(x):
    return np.sin(x)


# We define the parameters we want to optimize:
optimization_problem_parameters = [
    {
        "name": "x",
        "category": "uniform",
        "search_space": {
            "low": 0,
            "high": 2 * np.pi,
        },
    }
]

# We launch the optimization
best_sample = minimize(f, optimization_problem_parameters, number_of_evaluation=50)

print(best_sample["x"], 3 * np.pi / 2)

In [ ]:
from utils import MetaHeuristic
from benderopt import minimize


class BenderoptHeuristic(MetaHeuristic):
    def __init__(self, c, threshold=None):
        super().__init__(c, threshold)
        self.optimization_problem_parameters = []

    def f(self, **kwargs):
        x = [kwargs[f"x_{i}"] for i in range(self.c.OBJ_NUM)]
        if np.sum(x) > self.c.KNOWN_OBJECT_NUM:
            return np.sum(x)
        return -self.evaluate_selection(x)

    def strategy(self):
        for i in range(self.c.OBJ_NUM):
            self.optimization_problem_parameters.append(
                {
                    "name": f"x_{i}",
                    "category": "categorical",
                    "search_space": {"values": [0, 1]},
                }
            )
        res = minimize(
            self.f, self.optimization_problem_parameters, number_of_evaluation=100
        )
        if (
            np.sum([res[f"x_{i}"] for i in range(self.c.OBJ_NUM)])
            > self.c.KNOWN_OBJECT_NUM
        ):
            return np.zeros(self.c.OBJ_NUM)
        return [res[f"x_{i}"] for i in range(self.c.OBJ_NUM)]

In [ ]:
from config import Config

c = Config()
sa = BenderoptHeuristic(c)
mean, std = sa.evaluate_strategy(1)
print(f"Mean: {mean}, std: {std}")

In [ ]:
!pip install scikit-optimize

In [ ]:
from skopt import forest_minimize
from skopt.space import Categorical


class SkoptHeuristic(MetaHeuristic):
    def __init__(self, c, threshold=None):
        super().__init__(c, threshold)

    def f(self, x):
        if np.sum(x) > self.c.KNOWN_OBJECT_NUM:
            return np.sum(x)
        return -self.evaluate_selection(x)

    def strategy(self):
        initial = self._get_initial_selection()
        space = Categorical([0, 1], name="obj")
        res = forest_minimize(
            self.f,
            [space for _ in range(self.c.OBJ_NUM)],
            x0=list(initial),
            y0=self.f(initial),
        )
        if res.fun == 0:
            return np.zeros(self.c.OBJ_NUM)
        return res.x

In [ ]:
c = Config()
sa = SkoptHeuristic(c)
mean, std = sa.evaluate_strategy(1)
print(f"Mean: {mean}, std: {std}")

In [ ]:
!pip install rbfopt

In [ ]:
import rbfopt


class RbfoptHeuristic(MetaHeuristic):
    def __init__(self, c, threshold=None):
        super().__init__(c, threshold)

    def f(self, x):
        if np.sum(x) > self.c.KNOWN_OBJECT_NUM:
            return np.sum(x)
        return -self.evaluate_selection(x)

    def strategy(self):
        bb = rbfopt.RbfoptUserBlackBox(
            self.c.OBJ_NUM,
            np.array([0] * self.c.OBJ_NUM),
            np.array([1] * self.c.OBJ_NUM),
            np.array(["I"] * self.c.OBJ_NUM),
            self.f,
        )
        settings = rbfopt.RbfoptSettings(
            max_clock_time=1,
        )
        alg = rbfopt.RbfoptAlgorithm(settings, bb)
        val, x, itercount, evalcount, fast_evalcount = alg.optimize()
        if np.sum(x) > self.c.KNOWN_OBJECT_NUM:
            return np.zeros(self.c.OBJ_NUM)
        return x

In [ ]:
config = Config()
sa = RbfoptHeuristic(config)
mean, std = sa.evaluate_strategy(1)
print(f"Mean: {mean}, std: {std}")

In [ ]:
!pip install blackboxopt

In [ ]:
import parameterspace as ps
import blackboxopt as bbo
from blackboxopt.optimizers.space_filling import SpaceFilling
from blackboxopt.optimization_loops.sequential import run_optimization_loop

from utils import MetaHeuristic


class BlackboxoptHeuristic(MetaHeuristic):
    def __init__(self, c, threshold=None):
        super().__init__(c, threshold)

    def f(self, eval_spec):
        x = [eval_spec.configuration[f"obj_{i}"] for i in range(self.c.OBJ_NUM)]
        objectives = {"num_objects": self.evaluate_selection(x)}
        constraints = {"num_objects": self.c.KNOWN_OBJECT_NUM - np.sum(x)}

        return eval_spec.create_evaluation(
            objectives=objectives, constraints=constraints
        )

    def strategy(self):
        space = ps.ParameterSpace()
        for i in range(self.c.OBJ_NUM):
            space.add(ps.CategoricalParameter(f"obj_{i}", (0, 1)))
        optimizer = SpaceFilling(
            search_space=space,
            objectives=[
                bbo.Objective(name="num_objects", greater_is_better=True),
            ],
        )
        evaluations = run_optimization_loop(
            optimizer=optimizer,
            evaluation_function=self.f,
            timeout_s=5,
        )
        # filter out the configurations that violate the constraints
        valid_evaluations = [
            e for e in evaluations if e.constraints["num_objects"] >= 0
        ]
        print(f"Valid evaluations: {len(valid_evaluations)}")
        best = max(valid_evaluations, key=lambda e: e.objectives["num_objects"])
        if (
            np.sum([best[f"obj_{i}"] for i in range(self.c.OBJ_NUM)])
            > self.c.KNOWN_OBJECT_NUM
        ):
            return np.zeros(self.c.OBJ_NUM)
        return [best[f"obj_{i}"] for i in range(self.c.OBJ_NUM)]

In [ ]:
from config import Config

config = Config()
bboh = BlackboxoptHeuristic(config)
mean, std = bboh.evaluate_strategy(1)
print(f"Mean: {mean}, std: {std}")

In [ ]:
!pip install nevergrad

In [ ]:
import nevergrad as ng

from utils import MetaHeuristic


class NevergradHeuristic(MetaHeuristic):
    def __init__(self, c, threshold=None):
        super().__init__(c, threshold)

    def f(self, x):
        x = list(x[f"obj_{i}"] for i in range(self.c.OBJ_NUM))
        if np.sum(x) > self.c.KNOWN_OBJECT_NUM:
            return float(np.sum(x))
        return float(-self.evaluate_selection(x))

    def strategy(self):
        param = ng.p.Dict(
            **{f"obj_{i}": ng.p.Choice([0, 1]) for i in range(self.c.OBJ_NUM)}
        )
        optimizer = ng.optimizers.PortfolioDiscreteOnePlusOne(
            parametrization=param, budget=1000
        )
        for _ in range(optimizer.budget):
            x = optimizer.ask()
            loss = self.f(*x.args, **x.kwargs)
            print(f"Loss: {loss}")
            optimizer.tell(x, loss)
        final_recommendation = optimizer.provide_recommendation().value
        final_recommendation = list(
            final_recommendation[f"obj_{i}"] for i in range(self.c.OBJ_NUM)
        )
        if np.sum(final_recommendation) > self.c.KNOWN_OBJECT_NUM:
            return np.zeros(self.c.OBJ_NUM)
        return final_recommendation

In [ ]:
from config import Config

config = Config()
ngh = NevergradHeuristic(config)
mean, std = ngh.evaluate_strategy(1)
print(f"Mean: {mean}, std: {std}")

In [ ]:
!pip install pyswarms

In [ ]:
import numpy as np
from pyswarms.discrete import BinaryPSO

from utils import MetaHeuristic


class SwarmHeuristic(MetaHeuristic):
    def __init__(self, c, threshold=None):
        super().__init__(c, threshold)

    def f(self, x):
        res = []
        for i in range(x.shape[0]):
            if np.sum(x[i]) > self.c.KNOWN_OBJECT_NUM:
                res.append(np.sum(x[i]))
            else:
                res.append(-self.evaluate_selection(x[i]))
        return res

    def strategy(self):
        num_particles = 15
        options = {"c1": 2, "c2": 2, "w": 0.9, "k": num_particles, "p": 2}
        optimizer = BinaryPSO(
            n_particles=num_particles, dimensions=self.c.OBJ_NUM, options=options
        )
        cost, pos = optimizer.optimize(self.f, iters=2000)
        if np.sum(pos) > self.c.KNOWN_OBJECT_NUM:
            return np.zeros(self.c.OBJ_NUM)
        return pos

In [ ]:
from config import Config

config = Config()
sh = SwarmHeuristic(config)
mean, std = sh.evaluate_strategy(10)
print(f"Mean: {mean}, std: {std}")

In [ ]:
print(f"Mean: {mean}, std: {std}")
print(f"Time: {sh.get_mean_time()}")

In [ ]:
!pip install mealpy

In [ ]:
from mealpy import Problem
from typing import Union, List, Tuple


class ObjectProblem(Problem):
    def __init__(self, bounds=None, minmax="max", heuristic=None, **kwargs):
        self.heuristic = heuristic
        super().__init__(bounds, minmax, **kwargs)

    def obj_func(self, x: np.ndarray) -> Union[List, Tuple, np.ndarray, int, float]:
        x_decoded = self.decode_solution(x)
        x = x_decoded["object_selection"]

        if np.sum(x) > self.heuristic.c.KNOWN_OBJECT_NUM:
            return -np.sum(x)
        return self.heuristic.evaluate_selection(x)

In [ ]:
import mealpy

from al.utils import MetaHeuristic


class MealpyHeuristic(MetaHeuristic):
    def __init__(self, c, threshold=None):
        super().__init__(c, threshold)

    def f(self, x):
        if np.sum(x) > self.c.KNOWN_OBJECT_NUM:
            return -np.sum(x)
        return self.evaluate_selection(x)

    def strategy(self):
        bounds = mealpy.IntegerVar(
            lb=[
                0,
            ]
            * self.c.OBJ_NUM,
            ub=[
                1,
            ]
            * self.c.OBJ_NUM,
            name="object_selection",
        )
        problem = ObjectProblem(
            bounds=bounds, heuristic=self, minmax="max", log_to=None
        )
        optimizer = mealpy.get_optimizer_by_name("OriginalWOA")()
        termination = mealpy.Termination(max_fe=self.c.MH_BUDGET)
        optimizer.solve(problem, termination=termination)
        final = optimizer.problem.decode_solution(optimizer.g_best.solution)[
            "object_selection"
        ]
        return final

In [ ]:
from config import Config

config = Config()
mealpyh = MealpyHeuristic(config)
mean, std = mealpyh.evaluate_strategy(1)
print(f"Mean: {mean}, std: {std}")

In [ ]:
!pip install discrete-optimization

In [ ]:
!pip install scikit-opt

In [ ]:
from sko.GA import GA

demo_func = lambda x: (x[0] - 1) ** 2 + (x[1] - 0.05) ** 2 + x[2] ** 2
ga = GA(
    func=demo_func,
    n_dim=3,
    max_iter=500,
    lb=[-1, -1, -1],
    ub=[5, 1, 1],
    precision=[2, 1, 1e-7],
)
best_x, best_y = ga.run()
print("best_x:", best_x, "\n", "best_y:", best_y)

In [ ]:
from sko.GA import GA
import numpy as np

from utils import MetaHeuristic


class SkoHeuristic(MetaHeuristic):
    def __init__(self, c, threshold=None):
        super().__init__(c, threshold)

    def f(self, x):
        if np.sum(x) > self.c.KNOWN_OBJECT_NUM:
            return np.sum(x)
        return -self.evaluate_selection(x)

    def strategy(self):
        ga = GA(
            func=self.f,
            n_dim=self.c.OBJ_NUM,
            max_iter=250,
            lb=[
                0,
            ]
            * self.c.OBJ_NUM,
            ub=[
                1,
            ]
            * self.c.OBJ_NUM,
            precision=[
                1,
            ]
            * self.c.OBJ_NUM,
        )
        best_x, best_y = ga.run()
        if np.sum(best_x) > self.c.KNOWN_OBJECT_NUM:
            return np.zeros(self.c.OBJ_NUM)
        return best_x

In [ ]:
from config import Config

config = Config()
skoh = SkoHeuristic(config)
mean, std = skoh.evaluate_strategy(1)
print(f"Mean: {mean}, std: {std}")